In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

from tools import *
import numpy as np
import pickle

In [2]:
PATH_TO_LVK_DATA = '/Users/ulyanadupletsa/Documents/Paper2_Review/GWTC_data/'
PATH_TO_POSTERIORS_AVERAGED = '/Users/ulyanadupletsa/Documents/Paper2_Review/results/posterior_samples_averaged/'
PATH_TO_POSTERIORS_m1m2 = '/Users/ulyanadupletsa/Documents/Paper2_Review/results/posterior_samples_averaged_uniform_in_m12_prior/'
PATH_TO_GWFISH_AVERAGED = '/Users/ulyanadupletsa/Documents/Paper2_Review/results/gwfish_averaged/'
PATH_TO_INFO = '/Users/ulyanadupletsa/Documents/Paper2_Review/results/info/'
PATH_TO_LVK_SAMPLES = '/Users/ulyanadupletsa/Documents/Paper2_Review/results/posterior_samples/lvk_samples_snr/'

In [3]:
waveform = 'IMRPhenomXPHM'
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [4]:
events_list = np.loadtxt(PATH_TO_INFO + 'events_wf_median_priors_%s.txt' %waveform, dtype=str)
with open(PATH_TO_INFO + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [5]:
events_list[0]

'IGWN-GWTC3p0-v2-GW191126_115259_PEDataRelease_mixed_cosmo'

In [6]:
with open(PATH_TO_INFO + 'lvk_samples_len_%s.pkl' %waveform, 'rb') as f:
    samples_len = pickle.load(f)

In [7]:
lbs_errs = ['snr', 'chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn',
        'psi', 'phase', 'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl',
        'network_optimal_snr',
        'err_chirp_mass', 'err_mass_ratio', 'err_luminosity_distance', 'err_dec', 'err_ra',
        'err_theta_jn', 'err_psi', 'err_phase', 'err_geocent_time', 'err_a_1', 'err_a_2', 'err_tilt_1',
        'err_tilt_2', 'err_phi_12', 'err_phi_jl', 'err_sky_location']

In [ ]:
np.random.seed(42) # for reproducibility
rng = np.random.default_rng()

for event in events_list:
    label_err = get_label(detectors, event,'averaged', 0, 'errors')
    label_cov = get_label(detectors, event, 'averaged', 0, 'inv_fishers')
    data = pd.read_csv(PATH_TO_GWFISH_AVERAGED + event + '/' + label_err, names = lbs_errs, delimiter=' ', skiprows=1)
    for i in range(len(data)):
        mns = data[params].iloc[i].to_numpy()
        cov = np.load(PATH_TO_GWFISH_AVERAGED + event + '/' + label_cov)[i, :, :]
        samples = rng.multivariate_normal(mns, cov, samples_len[event])
        #mysamples = my_multivariate_normal(mns, cov, samples_len[event])
        samples_df = pd.DataFrame(samples, columns = params)
        samples_df.to_hdf(PATH_TO_POSTERIORS_AVERAGED + 'fisher_samples/fisher_samples_%s_%s.hdf5' %(event, i), mode='w', key='root')

In [8]:
with open(PATH_TO_INFO + 'chirp_mass_priors_%s.pkl' %waveform, 'rb') as f:
    chirp_mass_priors = pickle.load(f)

In [9]:
with open(PATH_TO_INFO + 'geocent_time_priors_%s.pkl' %waveform, 'rb') as f:
    geocent_time_priors = pickle.load(f)

In [10]:
# Build priors range dictionary
my_priors = {}
for event in events_list:
    event_dict = {
        'chirp_mass': np.array([float(chirp_mass_priors[event][0]), float(chirp_mass_priors[event][1])]),
        'mass_ratio': np.array([0.05, 1.0]),
        'luminosity_distance': np.array([10, 10000]),
        'dec': np.array([-np.pi/2, np.pi/2]),
        'ra': np.array([0, 2*np.pi]),
        'theta_jn': np.array([0, np.pi]),
        'psi': np.array([0, np.pi]),
        'phase': np.array([0, 2*np.pi]),
        'geocent_time': np.array([float(geocent_time_priors[event][0]), float(geocent_time_priors[event][1])]),
        'a_1': np.array([0, 0.99]),
        'a_2': np.array([0, 0.99]),
        'tilt_1': np.array([0, np.pi]),
        'tilt_2': np.array([0, np.pi]),
        'phi_12': np.array([0, 2*np.pi]),
        'phi_jl': np.array([0, 2*np.pi])
    }
    my_priors[event] = event_dict

In [11]:
min_array = np.array([-np.inf, 0, 0, -np.pi/2, 0, 0, 0, 0, -np.inf, 0, 0, 0, 0, 0, 0])
max_array = np.array([np.inf, 1, 20000, np.pi/2, 2*np.pi, np.pi, np.pi, 2*np.pi, np.inf, 1, 1, np.pi, np.pi, 2*np.pi, 2*np.pi])

for event in events_list:
    label_err = get_label(detectors, event,'averaged', 0, 'errors')
    label_cov = get_label(detectors, event, 'averaged', 0, 'inv_fishers')
    data = pd.read_csv(PATH_TO_GWFISH_AVERAGED + event + '/' + label_err, names = lbs_errs, delimiter=' ', skiprows=1)
    for i in range(len(data)):
        mns = data[params].iloc[i].to_numpy()
        cov = np.load(PATH_TO_GWFISH_AVERAGED + event + '/' + label_cov)[i, :, :]
        samples_tmvn = get_samples_from_TMVN(min_array, max_array, mns, cov, samples_len[event])
        new_df = pd.DataFrame(samples_tmvn.T, columns = params)
        if event == 'IGWN-GWTC3p0-v2-GW200322_091133_PEDataRelease_mixed_cosmo':
            my_priors[event]['mass_ratio'] = np.array([0.02, 1.0])
        posteriors = get_posteriors(new_df, my_priors[event], samples_len[event])
        posteriors_df = pd.DataFrame(posteriors, columns = params)
        posteriors_df.to_hdf(PATH_TO_POSTERIORS_m1m2 + 'fisher_plus_priors_samples/fisher_plus_priors_samples_%s_%s.hdf5' %(event, i), mode='w', key='root')